In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw2.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 2: Decision trees and machine learning fundamentals 
### Associated lectures: [Lectures 2 and 3](https://ubc-cs.github.io/cpsc330-2024W1/README.html) 

**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html)**. 

Imports
------

In [2]:
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["font.size"] = 16

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You are welcome to broadly discuss questions with your classmates but your final answers must be your own. **We are not allowing group submission for this homework assignment.**  

**Important**: In this assignment, we are using an autograder so you can receive feedback on your solutions. For the autograder to work, you need to ensure that `otter-grader` is installed in the course environment. If you successfully installed the course environment, `otter-grader` should already be included. If not, you'll need to install it explicitly.

1. Go to the command line/terminal.
2. Activate the course conda environment and install otter-grader using the following commands.

    ```
    conda activate cpsc330
    pip install otter-grader
    ```

The cell at the top of the notebook will throw an error if you do not install `otter-grader`. Once you are done with the assignment follow the instructions below before the submission. 

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/master/docs/homework_instructions.md). 

1. Before submitting the assignment, run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Follow the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2024W1/docs/homework_instructions.html), which include information on how to do your assignment and how to submit your assignment.
4. Upload your solution on Gradescope. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
5. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope.
   
</div>

_Points:_ 3

<!-- END QUESTION -->

<br><br>

<div class="alert alert-info">
    
## Worksheet section

</div>

## Exercise 1: Terminology
rubric={autograde}

**Your Task:**

Please fill in each of the following sentences using the provided machine learning terminologies from the list below. Keep in mind that each term should be used only once!

**List of Terms (in no particular order):**

    a) hyperparameters
    b) training
    c) tree depth
    d) parameters
    e) example/data point
    f) features
    g) target
    h) root node
    i) branch
    j) leaf node

    

1. In the context of working with data, each individual row or instance, which includes both feature values and the corresponding target, is commonly referred to as an ________.

2. In supervise machine learning, the ________ is the variable we aim to predict or understand.

3. Before diving into the modeling process, it is necessary to define specific settings that impact the learning process; these settings are known as ________.

4. After the model has completed the training phase, it acquires specific values, such as which features to prioritize and the threshold for splitting them in the case of decision trees; these acquired values are referred to as _________.
   
5. In decision trees, the initial question we ask, which serves as the starting point, is commonly referred to as the ________.

6. The total number of steps or transitions from the initial question all the way to the final prediction in a decision tree is known as the ________.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 3

In [3]:
# format your answer like this: terminology = ['x','x','x','x','x','x']
terminology = ['e','g','a','d','h','c']

...

Ellipsis

In [4]:
grader.check("q1.1")

q1.1 results: All test cases passed!

<br><br>

## Exercise 2: Decision trees with a toy dataset 
<hr>

Suppose you have three different job offers with comparable salaries and job descriptions. You want to decide which one to accept, and you want to make this decision based on which job is likely to make you happy. Being a very systematic person, you come up with three features associated with the offers, which are important for your happiness: whether the colleagues are supportive, whether there is work-hour flexibility, and whether the company is a start-up or not. So the `X` of your offer data looks as follows: 

In [5]:
offer_data = {
    # Features
    "supportive_colleagues": [1, 0, 0, 1],
    "work_hour_flexibility": [0, 0, 1, 1],
    "start_up": [0, 1, 1, 1],    
}

offer_df = pd.DataFrame(offer_data)
offer_df

,supportive_colleagues,work_hour_flexibility,start_up
0,1,0,0
1,0,0,1
2,0,1,1
3,1,1,1


Your goal is to get predictions for these rows. In other words, for each row, you want to predict whether that job would make you **happy** or **unhappy**.   

So you ask the following questions to some of your friends (who you think have similar notions of happiness) regarding their jobs:

1. Do you have supportive colleagues? (1 for 'yes' and 0 for 'no')
2. Do you have flexible work hours? (1 for 'yes' and 0 for 'no')
3. Do you work for a start-up? (1 for 'start up' and 0 for 'non start up')
4. Are you happy in your job? (happy or unhappy)

Suppose you get the following data from this toy survey. You decide to train a machine learning model using this toy survey data and use this model to predict which job from `offer_df` is likely to make you happy. 

In [6]:
import pandas as pd

happiness_data = {
    # Features
    "supportive_colleagues": [1, 1, 1, 0, 0, 1, 1, 0, 1, 0],
    "work_hour_flexibility": [1, 1, 0, 1, 1, 0, 1, 0, 0, 0],
    "start_up": [1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
    # Target
    "target": [
        "happy",
        "happy",
        "happy",
        "unhappy",
        "unhappy",
        "happy",
        "happy",
        "unhappy",
        "unhappy",
        "unhappy",
    ],
}

train_df = pd.DataFrame(happiness_data)
train_df

,supportive_colleagues,work_hour_flexibility,start_up,target
0,1,1,1,happy
1,1,1,0,happy
2,1,0,1,happy
3,0,1,0,unhappy
4,0,1,1,unhappy
5,1,0,0,happy
6,1,1,0,happy
7,0,0,1,unhappy
8,1,0,1,unhappy
9,0,0,0,unhappy


<br><br>

### 2.1 Decision stump by hand 
rubric={autograde}

**Your tasks:**

- With this toy dataset, build a decision stump (decision tree with only 1 split) manually, splitting on the condition `supportive_colleagues <= 0.5`. What training accuracy would you get with this decision stump? Save the accuracy as a decimal in an object named `supportive_colleagues_acc`. 

> You do not have to show any calculations or code. 

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 1

In [7]:
supportive_colleagues_acc = 0.9

...

Ellipsis

In [8]:
grader.check("q2.1")

q2.1 results: All test cases passed!

<br><br>

### 2.2 Separating features and target
rubric={autograde}

Recall that in `scikit-learn`, before building a classifier, we need to separate features and target. 

**Your tasks:**

1. Separate features and target from `train_df` and save them in `X_train_toy` and `y_train_toy`, respectively. 

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 1

In [9]:
X_train_toy = train_df[["supportive_colleagues", "work_hour_flexibility", "start_up"]]
y_train_toy = train_df["target"]


In [10]:
grader.check("q2.2")

q2.2 results: All test cases passed!

<br><br>

### 2.3 Create a decision tree classifier object
rubric={autograde}

**Your tasks:**

1. Create a `DecisionTreeClassifier` object with `random_state=16` and store it in a variable called `toy_tree`.

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 1

In [11]:
toy_tree = DecisionTreeClassifier(random_state=16)

...

Ellipsis

In [12]:
grader.check("q2.3")

q2.3 results: All test cases passed!

<br><br>

### 2.4 `fit` the decision tree classifier 
rubric={autograde}

**Your tasks:**

1. Now train a decision tree model by calling `fit` on `toy_tree` with `X_train_toy` and `y_train_toy` created above. 

<div class="alert alert-warning">

Solution_2.4
    
</div>

_Points:_ 1

In [13]:
toy_tree.fit(X_train_toy, y_train_toy)

DecisionTreeClassifier(random_state=16)

In [14]:
grader.check("q2.4")

q2.4 results: All test cases passed!

<br><br>

### 2.5 Visualize the trained decision tree
rubric={autograde}


**Your tasks:**
- Visualize the trained decision tree model using the [`tree.plot_tree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) method in `sklearn` by passing the appropriate values for the following arguments: 
    - `feature_names`
    - `class_names`
  
Save the names of the features in `feature_names` variable, names of the classes in `class_names` variable and the visualization tree returned by the function in a variable called `toy_tree_viz`.

<div class="alert alert-warning">

Solution_2.5
    
</div>

_Points:_ 2

In [15]:
feature_names = ["supportive_colleagues", "work_hour_flexibility", "start_up"] # feature names 
class_names = ["happy", "unhappy"] # unique class names 
toy_tree_viz = tree.plot_tree(toy_tree, feature_names= feature_names, class_names= class_names)


In [16]:
grader.check("q2.5")

q2.5 results: All test cases passed!

<br><br>

### 2.6 Depth of the tree
rubric={autograde}

**Your tasks:**

1. What's the depth of the learned decision tree model? Save it as an integer in the variable `toy_depth` below. 

<div class="alert alert-warning">

Solution_2.6
    
</div>

_Points:_ 1

In [17]:
toy_depth = 3

...

Ellipsis

In [18]:
grader.check("q2.6")

q2.6 results: All test cases passed!

<br><br>

### 2.7 Accuracy calculation
rubric={autograde}

**Your tasks:**

1. Evaluate the `toy_tree` on the training data (i.e., call `score()` on `X_train_toy` and `y_train_toy`) and store the score in a variable called `train_acc`.

<div class="alert alert-warning">

Solution_2.7
    
</div>

_Points:_ 1

In [19]:
train_acc = 0.9
toy_tree.score(X_train_toy, y_train_toy)



0.9

In [20]:
grader.check("q2.7")

q2.7 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 2.8 Discussion
rubric={reasoning}

**Your tasks:**

1. Do you achieve perfect training accuracy? If so, what are the reasons behind this, and if not, what factors contribute to the imperfection?

<div class="alert alert-warning">

Solution_2.8
    
</div>

_Points:_ 2

_I achieved an accuracy score of 0.9, not a perfect 1. This is because there was one person that had supportive colleagues but was still unhappy. If we had a larger dataset we may be able to indentify more patterns and thus add more depth levels, resulting in a more accurate model._ 

<!-- END QUESTION -->

<br><br>

### 2.9 Predicting on the offer data 
rubric={autograde}

**Your tasks:**

1. Using the trained decision tree above, predict the targets for all examples in `offer_df` and store them as a list in the `predictions` variable below.
2. In which jobs you are likely to be happy? Append the index or indices of all the examples where you are likely to be happy to the 'happy_job_indices' list below.

In [21]:
offer_df

,supportive_colleagues,work_hour_flexibility,start_up
0,1,0,0
1,0,0,1
2,0,1,1
3,1,1,1


<div class="alert alert-warning">

Solution_2.9
    
</div>

_Points:_ 2

In [22]:
predictions = toy_tree.predict(offer_df)
predictions = predictions.tolist()
happy_job_indices = [0, 3]


In [23]:
print(predictions) ##thus you are likely to be happy in job 0 and job 3
predictions.count('happy')

['happy', 'unhappy', 'unhappy', 'happy']


2

In [24]:
grader.check("q2.9")

q2.9 results: All test cases passed!

<br><br><br><br>

<div class="alert alert-info">
    
## Homework Section

</div>

## Exercise 3: Decision trees on Spotify Song Attributes dataset 
<hr>

### Introducing the dataset
 
For the rest of the homework, you'll be using Kaggle's [Spotify Song Attributes](https://www.kaggle.com/geomack/spotifyclassification/home) dataset. The dataset contains a number of features of songs from 2017 and a binary variable `target` that represents whether the user liked the song (encoded as 1) or not (encoded as 0). See the documentation of all the features [here](https://developer.spotify.com/documentation/web-api/reference/get-audio-features). 

This dataset is publicly available on Kaggle, and you will have to download it yourself. Follow the steps below to get the data CSV. 

1. If you do not have an account with [Kaggle](https://www.kaggle.com/), you will first need to create one (it's free).
2. Login to your account and [download](https://www.kaggle.com/geomack/spotifyclassification/download) the dataset.
3. Unzip the data file if needed, then rename it to `spotify.csv`, and move it under the `data` directory.

> You will not be able to push it to your repository (hopefully) because I have seeded the repository with `.gitignore`. 

### 3.1 Reading the data CSV
rubric={autograde}
 
**Your tasks:**
1. Read in the data CSV and store it as a pandas dataframe named `spotify_df`. The first column of the .csv file should be set as the index.

> Make sure you have put the data CSV as `spotify.csv` under the data directory (data/spotify.csv). When you read the data file, use this relative path for the autograder to work properly on Gradescope. 

<div class="alert alert-warning">

Solution_3.1
    
</div>

_Points:_ 2

In [25]:
spotify_df = pd.read_csv("data/spotify.csv", index_col = 0)

spotify_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,0.00106,0.584,274404,0.932,0.002690,1,0.1290,-3.501,1,0.3330,74.976,4.0,0.211,0,Like A Bitch - Kill The Noise Remix,Kill The Noise
2013,0.08770,0.894,182182,0.892,0.001670,1,0.0528,-2.663,1,0.1310,110.041,4.0,0.867,0,Candy,Dillon Francis
2014,0.00857,0.637,207200,0.935,0.003990,0,0.2140,-2.467,1,0.1070,150.082,4.0,0.470,0,Habit - Dack Janiels & Wenzday Remix,Rain Man
2015,0.00164,0.557,185600,0.992,0.677000,1,0.0913,-2.735,1,0.1330,150.011,4.0,0.623,0,First Contact,Twin Moons


In [26]:
grader.check("q3.1")

q3.1 results: All test cases passed!

<br><br>

### 3.2 Data splitting 
rubric={autograde}

**Your tasks:**

1. Split the dataframe into `train_df` and `test_df` with `random_state=123` and `test_size=0.2`. 


<div class="alert alert-warning">

Solution_3.2
    
</div>

_Points:_ 2

In [27]:
train_df, test_df = train_test_split(spotify_df, random_state=123, test_size=0.2)

...

Ellipsis

In [28]:
grader.check("q3.2")

q3.2 results: All test cases passed!

<br><br>

### 3.3 Number of training and test examples
rubric={autograde}

**Your tasks:**
1. How many training and test examples do we have? Store them as integers in the variables below. 

<div class="alert alert-warning">

Solution_3.3
    
</div>

_Points:_ 1

In [29]:
# Please provide integer values
n_train_samples = 1613
n_test_samples = 404

len(train_df)
len(test_df)

404

In [30]:
grader.check("q3.3")

q3.3 results: All test cases passed!

<br><br>

### 3.4 `describe` method 
rubric={autograde}

**Your tasks:**

1. Store the output of `describe()` **on the training split** in `spotify_summary` variable below and display the summary statistics. By default, this function will compute some summary statistics of the numeric columns.

> Note that `describe` returns another DataFrame.

<div class="alert alert-warning">

Solution_3.4
    
</div>

_Points:_ 2

In [31]:
spotify_summary = train_df.describe()

spotify_summary

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
count,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000,1613.000000
mean,0.185627,0.616745,247114.827650,0.681296,0.136862,5.383137,0.189189,-7.112929,0.621203,0.091277,121.979777,3.964662,0.497587,0.507750
std,0.259324,0.163225,81177.300308,0.211612,0.277744,3.620422,0.153170,3.838867,0.485238,0.087890,26.965641,0.255201,0.247378,0.500095
min,0.000005,0.122000,16042.000000,0.014800,0.000000,0.000000,0.018800,-33.097000,0.000000,0.023100,47.859000,1.000000,0.035900,0.000000
25%,0.009190,0.511000,200105.000000,0.564000,0.000000,2.000000,0.092300,-8.388000,0.000000,0.037300,100.518000,4.000000,0.295000,0.000000
50%,0.062500,0.629000,230200.000000,0.714000,0.000071,6.000000,0.127000,-6.248000,1.000000,0.054900,121.990000,4.000000,0.496000,1.000000
75%,0.251000,0.738000,272533.000000,0.844000,0.057300,9.000000,0.243000,-4.791000,1.000000,0.107000,137.932000,4.000000,0.690000,1.000000
max,0.995000,0.984000,849960.000000,0.997000,0.976000,11.000000,0.969000,-0.307000,1.000000,0.816000,219.331000,5.000000,0.992000,1.000000


In [32]:
grader.check("q3.4")

q3.4 results: All test cases passed!

<br><br>

### 3.5 Largest range feature
rubric={autograde}

**Your tasks:**

1. Which feature has the largest range? Store the feature name as a string in the `largest_range_feature` variable below.

> Hint: You can subtract the min value from the max value of the column to get the range.

<div class="alert alert-warning">

Solution_3.5
    
</div>

_Points:_ 2

In [33]:
largest_range_feature = "duration_ms"


In [34]:
grader.check("q3.5")

q3.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 3.6 Plotting histograms 
rubric={viz}

The starter code below produces histograms for the `loudness` feature using pandas plotting. The histograms show the distribution of the feature values in the training set, separated for positive (target=1, i.e., user liked the song) and negative (target=0, i.e., user disliked the song) examples. There are two different histograms, one for target = 0 and one for target = 1, and they are overlaid on top of each other. The histogram shows that extremely quiet songs tend to be disliked (more blue bars than orange on the left) and very loud songs also tend to be disliked (more blue than orange on the far right).

> Note: I am using `matplotlib` and pandas plotting here. If you decide to use other visualization libraries, they might not work on Gradescope because they won't be present in the environment file on Gradescope. 


In [35]:
feat = "loudness"
train_df.groupby("target")[feat].plot.hist(bins=50, alpha=0.5, legend=True, density = True, title = "Histogram of " + feat);
plt.xlabel(feat);

**Your tasks:**

Create histograms for the following features in the order below.
- acousticness
- danceability
- tempo
- energy
- valence

> To adhere to the [DRY (Don't Repeat Yourself)](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) principle, make sure you use a `for` loop for your plotting, rather than repeating the plotting code 4 times. For this to work, I used `plt.show()` at the end of your loop, which draws the figure and resets the canvas for your next plot.

<div class="alert alert-warning">

Solution_3.6
    
</div>

_Points:_ 3

In [36]:
feats = ["acousticness", "danceability", "tempo", "energy", "valence"]

for feat in feats:
    train_df.groupby("target")[feat].plot.hist(bins=5, alpha=0.5, legend=True, density = True, title = "Histogram of " + feat);
    plt.xlabel(feat);
    plt.show()

2024-10-02 21:35:15.832 python[27827:5539340] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-02 21:35:15.832 python[27827:5539340] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.7 Identical histograms
rubric={reasoning}

**Your tasks:**

1. Let's say that, for a particular feature, the histograms of that feature are identical for the two target classes. Does that mean the feature is not useful for predicting the target class? Briefly explain. 

<div class="alert alert-warning">

Solution_3.7
    
</div>

_Points:_ 2

_A feature with identical histograms for multiple target classes is not a useful predictor because having identical histograms suggests that the feature doesn't have a relevent impact on distinguishing information between the classes; thus it doesn't help in differentiating them._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 3.8 Which columns to include? 
rubric={reasoning}

Note that the dataset includes two text features labeled `song_title` and `artist`.

**Your tasks:**

1. Do you believe that these features could be valuable in predicting whether the user liked the song or not? If so, what makes them suitable, and if not, what makes them unsuitable?
2. Do you anticipate any challenges in using these features in their current form within your model? Please provide a brief explanation. 

<div class="alert alert-warning">

Solution_3.8
    
</div>

_Points:_ 3

1. 
   I think they could still be valuable in predicting the likeability of a song because some people may be more or less likely to enjoy a song if it is by an artist they support or son't support and some people may be more drawn to a song with a catchy title then a boring title. Thus, there may be a coorelation between these features and likability.

2. 
    The text in the current model is not uniform and would require some preprocessing to be used. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 4: Model building
<hr>

Now that we did some preliminary exploratory data analysis (EDA), let's move on to modeling. 

<br><br>

### 4.1 Creating `X` and `y`
rubric={autograde}

**Your tasks:**

1. Separate `X` and `y` from `train_df` and `test_df` from the previous exercise and store them as `X_train`, `y_train`, `X_test`, `y_test`, respectively. Skip the `song_title` and `artist` columns for now. 

<div class="alert alert-warning">

Solution_4.1
    
</div>

_Points:_ 2

In [37]:
X_train = train_df.drop(columns = ["target", "song_title", "artist"])
y_train = train_df["target"]
X_test = test_df.drop(columns = ["target", "song_title", "artist"])
y_test = test_df["target"]


In [38]:
grader.check("q4.1")

q4.1 results: All test cases passed!

<br><br>

<br><br>

### 4.2 The baseline model: `DummyClassifier`
rubric={autograde}

**Your tasks:**
1. Carry out 10-fold cross-validation using `DummyClassifier` with `random_state=123`. Store the mean cross-validation score in the `dummy_score` variable below. 

<div class="alert alert-warning">

Solution_4.2
    
</div>

_Points:_ 1

In [39]:
dummy = DummyClassifier(strategy='most_frequent', random_state=123)
dummy_scores = cross_val_score(dummy, X_train, y_train, cv=10)

dummy_score = dummy_scores.mean()

dummy_score

0.5077524729698643

In [40]:
grader.check("q4.2")

q4.2 results: All test cases passed!

<br><br>

### 4.3 Creating a Decision Tree model
rubric={autograde}

**Your tasks:**

1. Create a `DecisionTreeClassifier` with `random_state=123` and store it in a variable called `spotify_tree`.

<div class="alert alert-warning">

Solution_4.3
    
</div>

_Points:_ 1

In [41]:
spotify_tree = DecisionTreeClassifier(random_state=123)

...

Ellipsis

In [42]:
grader.check("q4.3")

q4.3 results: All test cases passed!

<br><br>

### 4.4 Cross-validation with `DecisionTreeClassifier`
rubric={autograde}

**Your tasks:** 

1. Carry out 10-fold cross validation with the `spotify_tree` object above using `cross_validate` on `X_train` and `y_train`. Pass `return_train_score=True` to `cross_validate`. Save the results as a pandas dataframe in a variable called `dt_scores_df`. 

<div class="alert alert-warning">

Solution_4.4
    
</div>

_Points:_ 4

In [43]:
dt_scores = cross_validate(spotify_tree, X_train, y_train, cv=10, return_train_score=True)
dt_scores_df = pd.DataFrame(dt_scores)

mean_test_score = dt_scores_df['test_score'].mean()
print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", dt_scores_df['train_score'].mean())

Mean Test Score: 0.6695383789586687
Mean Train Score: 0.9994489408843148


In [44]:
grader.check("q4.4")

q4.4 results:
    q4.4 - 1 result:
        ✅ Test case passed

    q4.4 - 2 result:
        ✅ Test case passed

    q4.4 - 3 result:
        ❌ Test case failed
        Trying:
            assert np.isclose(round(dt_scores_df['test_score'].mean(), 3), 0.671), 'Your test scores are incorrect'
        Expecting nothing
        **********************************************************************
        Line 1, in q4.4 2
        Failed example:
            assert np.isclose(round(dt_scores_df['test_score'].mean(), 3), 0.671), 'Your test scores are incorrect'
        Exception raised:
            Traceback (most recent call last):
              File "/Users/litabates/miniconda3/envs/cpsc330/lib/python3.12/doctest.py", line 1357, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q4.4 2[0]>", line 1, in <module>
                assert np.isclose(round(dt_scores_df['test_score'].mean(), 3), 0.671), 'Your test scores are incorrect'
            AssertionError: Your test scores are incorrect
        Trying:
            assert np.isclose(round(dt_scores_df['train_score'].mean(), 3), 0.999), 'Your train scores are incorrect'
        Expecting nothing
        ok

<br><br>

<!-- BEGIN QUESTION -->

### 4.5 Examining cross-validation scores
rubric={reasoning}

**Your tasks:** 
1. Inspect the 10 sub-scores from the 10 folds of cross-validation. To what extent do you trust the numerical value / precision of the cross validation score? Briefly explain.  
2. Do you see a significant difference between the training scores and the cross-validation scores? Briefly discuss in 1 to 2 sentences. 

<div class="alert alert-warning">

Solution_4.5
    
</div>

_Points:_ 4

1.
  Cross validation scores are relatively reliable because they provide a reference for average performance,
this means that the presence of outliers is not detrimental; however, they can be susceptible to overfitting 
or underfitting
2.
  There is about a 17% difference in the training score vs the test score which is significant. 
The lower training score could indicate that the model is not representing the optimal data pattern, while the
significantly higher cross-validation score could indicate a case of underfitting

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 5: Hyperparameters
<hr>

In this exercise, you'll experiment with the `max_depth` hyperparameter of the decision tree classifier. See the [`DecisionTreeClassifier` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for more details.

### 5.1 Train and cross-validation accuracies 
rubric={autograde}


**Your tasks:**

1. Explore the `max_depth` hyperparameter. Run 10-fold cross-validation for trees with the following values of `max_depth`: `np.arange(1, 25, 2)`. Set the `random_state` of `DecisionTreeClassifier` to 123 in each case for reproducibility. 
2. For each `max_depth`, get both the mean train accuracy and the mean cross-validation accuracy. Store your results in the `results_df` dataframe, where the max_depth is set as the index. 

> Note: generally speaking (for all assignments) you are welcome to copy/paste code directly from the lecture notes, though I ask that you add a small citation (e.g. "Adapted from lecture 1") if you do so.

In [45]:
depths = np.arange(1, 25, 2)
depths

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23])

<div class="alert alert-warning">

Solution_5.1
    
</div>

_Points:_ 6

In [46]:
results_df = pd.DataFrame(columns=['mean_train_accuracy', 'mean_cv_accuracy'])

for max_depth in depths:
    train_accuracy = DecisionTreeClassifier(max_depth=max_depth, random_state = 123).fit(X_train, y_train).score(X_train, y_train) 
    #adapted from lecture 3
    tree_cv = DecisionTreeClassifier(max_depth=max_depth, random_state=123)
    cv_accuracy = cross_val_score(tree_cv, X_train, y_train, cv=10).mean()
    results_df.loc[max_depth] = [train_accuracy, cv_accuracy]


In [47]:
results_df

,mean_train_accuracy,mean_cv_accuracy
1,0.650341,0.642926
3,0.726596,0.693766
5,0.788593,0.711705
7,0.855549,0.701791
9,0.912585,0.687505
11,0.946683,0.685016
13,0.979541,0.671379
15,0.994420,0.663944
17,0.999380,0.669527
19,0.999380,0.670159


In [48]:
results_df.shape

(12, 2)

In [49]:
grader.check("q5.1")

q5.1 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 5.2 Visualization 
rubric={viz}

1. Make a plot with `max_depth` on the *x*-axis and the train and cross-validation accuracies on the *y*-axis. That is, your plot should have two curves, one for train and one for cross-validation. 

**Ensure your plot includes the following:**

1. Both the train accuracy and the cross-validation accuracy are included in the plot.
2. Include a legend to specify which is which. 
3. The provided `max_depth` values are used.
4. The x-axis and y-axis have reasonable names.
5. The data points are correct.

<div class="alert alert-warning">

Solution_5.2
    
</div>

_Points:_ 5

In [50]:
plt.plot(depths, results_df["mean_train_accuracy"], color="pink", label="Train Accuracy")
plt.plot(depths, results_df["mean_cv_accuracy"], color="purple", label="CV Accuracy")

plt.show()

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 5.3 `max_depth` and the fundamental tradeoff
rubric={reasoning}

**Your tasks:**
1. Discuss how changing the `max_depth` hyperparameter affects the training and cross-validation accuracy. 

<div class="alert alert-warning">

Solution_5.3
    
</div>

_Points:_ 3

While the cross-validation score peaks at a depth of 5 and stays relatively low for the rest of the depths, the training accuracy gradually increases between 1 and 17 when it starts to plateau.

<!-- END QUESTION -->

<br><br>

### 5.4 Picking the "best" value for `max_depth`
rubric={autograde}

**Your tasks:**
1. From these results, pick the "best" `max_depth`, the one which gives the maximum cross-validation score. Store it in a variable called `best_max_depth` as an integer. 

<div class="alert alert-warning">

Solution_5.4
    
</div>

_Points:_ 2

In [51]:
best_max_depth = 5

...

Ellipsis

In [52]:
grader.check("q5.4")

q5.4 results: All test cases passed!

<br><br><br><br>

## Exercise 6: Final assessment
<hr>

Now that we have our finalized model, we are ready to evaluate it on the test set. 

### 6.1 Final assessment on the test split 
rubric={autograde}



**Your tasks:**

1. Create a decision tree model `best_model` using the `best_max_depth` you chose in the previous exercise with `random_state=123`. 
2. Fit the `best_model` on the _entire training set_ (`X_train` and `y_train`). 
2. Compute the test score (on `X_test` and `y_test`) and store it in a variable called `test_score` below. 

<div class="alert alert-warning">

Solution_6.1
    
</div>

_Points:_ 3

In [53]:
best_model = DecisionTreeClassifier(max_depth = best_max_depth, random_state=123)
best_model.fit(X_train, y_train)
test_score = best_model.score(X_test, y_test)

test_score

0.7178217821782178

<br><br>

<!-- BEGIN QUESTION -->

### 6.2 Analysis
rubric={reasoning}

**Your tasks:**

1. How do the test scores compare to the cross-validation scores? Briefly discuss. 
2. Why can't you simply pick the value of `max_depth` that gives the best accuracy on the training data? (Answer in maximum 2 to 3 sentences.)
3. Do you think that the `max_depth` you chose would generalize to other "spotify" datasets (i.e., data on other spotify users)?

<div class="alert alert-warning">

Solution_6.2
    
</div>

_Points:_ 4

1. While the test score is slightly higher than the highest cross-validation score they are still relatively similar, this means that the model is behaving as intended.
2. This can result in overfitting; the model will learn the specific patterns in the training data instead of the generalizable trends.
3. I think it may generalize well but due to the vast range of Spotify users it would still be important to test with other datasets to ensure accuracy.

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Congratulations on working with your first machine learning model! Well done 👏👏!

![](img/eva-well-done.png)